In [ ]:
import sys
import subprocess


def main():
    command = sys.argv[1]

    # Define test directories and corresponding coverage targets
    test_config = {
        'unit': {'dir': 'tests/unit_tests', 'cov': ['config', 'src']},
        'integration': {'dir': 'tests/integration_tests', 'cov': []},
        'component': {'dir': 'tests/component_tests', 'cov': []},
        'all': {'dir': 'tests', 'cov': ['config', 'etl']},
    }

    # Check to see if a command was supplied for the test run
    if command in test_config:
        # Access the test_config dictionary to get the test directory
        # and coverage targets
        test_dir = test_config[command]['dir']
        cov_sources = ','.join(test_config[command]['cov'])

        # Build the test command for tests with coverage
        if cov_sources:
            cov_command = (
                f'ENV=test coverage run --source={cov_sources} '
                f'--omit=*/__init__.py -m pytest --verbose {test_dir} '
                '&& coverage report -m && coverage html '
                '&& coverage report --fail-under=90'
            )
        else:
            cov_command = f'ENV=test pytest --verbose {test_dir}'

        subprocess.run(cov_command, shell=True)
    elif command == 'lint':
        subprocess.run(['flake8', '.'])
    else:
        raise ValueError(f"Unknown command: {command}")


if __name__ == "__main__":
    if len(sys.argv) < 2:
        raise ValueError(
            "Usage: run_tests.py <unit|integration|component|all|lint>"
        )
    else:
        main()


In [ ]:
import pytest
from unittest.mock import MagicMock, patch
from sqlalchemy.exc import OperationalError, SQLAlchemyError
from src.utils.db_utils import get_db_connection, DatabaseConnectionError


@pytest.fixture
def test_connection_params():
    return {
        "dbname": "test_db",
        "user": "test_user",
        "password": "test_password",
        "host": "testhost",
        "port": "1234",
    }


def test_get_db_connection_full_component_success(test_connection_params):
    """Test successful interaction between create_db_engine and
    get_db_connection components."""
    # Arrange
    mock_connection = MagicMock()
    mock_engine = MagicMock()
    mock_engine.connect.return_value = mock_connection

    with patch("src.utils.db_utils.create_db_engine", return_value=mock_engine) as mock_create_engine, \
         patch("src.utils.db_utils.logger") as mock_logger:
        # Act
        connection = get_db_connection(test_connection_params)

        # Assert
        mock_create_engine.assert_called_once_with(test_connection_params)
        mock_engine.connect.assert_called_once()
        assert connection == mock_connection
        # Verify both functions logged success
        assert mock_logger.info.call_count == 1

In [ ]:
import pandas as pd
import pytest
from src.extract.extract_customers import (
    extract_customers,
    TYPE,
    FILE_PATH,
    EXPECTED_PERFORMANCE,
)


@pytest.fixture
def mock_log_extract_success(mocker):
    return mocker.patch("src.extract.extract_customers.log_extract_success")


@pytest.fixture
def mock_logger(mocker):
    return mocker.patch("src.extract.extract_customers.logger")


def test_extract_customers_csv_to_dataframe(mocker):
    mock_df = pd.DataFrame(
        {"id": [1, 2, 3], "name": ["Alice", "Bob", "Charlie"]}
    )
    mocker.patch(
        "src.extract.extract_customers.pd.read_csv", return_value=mock_df
    )

    # Call the function
    df = extract_customers()

    # Assertions
    assert isinstance(df, pd.DataFrame)
    pd.testing.assert_frame_equal(df, mock_df)

